In [1]:
import pandas as pd
import gc
import sys
import os
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3
src_path = os.path.abspath(r'C:\Users\User\OneDrive - Imperial College London\Desktop\Documents\GitHub\Credit-Card\src')
sys.path.append(src_path)

from adtk.data import validate_series
from adtk.detector import ThresholdAD
from adtk.visualization import plot

from feature_engineering.encoding import group_low_frequency_categories, encode_categories, log_encode
from feature_engineering.new_features import calculate_time_delta, extract_bin, extract_check_digit, extract_datetime_component, average_transactions_per_time
from data.db_functions import query_data
from data.clean_data import update_dtypes

gc.collect()

0

In [2]:
db_dir = r'C:\Users\User\OneDrive - Imperial College London\Desktop\Documents\GitHub\Credit-Card\data\database (SQLite)\raw_data.db'

query = "SELECT * FROM raw_data WHERE business_id IN ('640fdf71d95a90ebc209167d')"

with sqlite3.connect(db_dir) as conn:
    df = pd.read_sql_query(query, conn)


df.head()
df.shape
df.dtypes

business_id                         object
auth_status                         object
eci                                 object
descriptor                          object
auth_failure_reason                 object
charge_failure_reason               object
amount                             float64
authorized_amount                  float64
mid_label                           object
bank_merchant_id                    object
capture_amount                     float64
created                             object
authentication_type                 object
card_bank                           object
card_brand                          object
card_expiration_month                int64
card_expiration_year                 int64
cavv                                object
country                             object
currency                            object
ip_address                          object
masked_card_number                  object
card_holder_name                    object
card_type  

In [3]:
df = update_dtypes(df)
df.dtypes

df = df.dropna(subset=['created'])

nan_counts = df.isna().sum()
print("NaN values in each column:")
print(nan_counts[nan_counts > 0])

C:\Users\User\OneDrive - Imperial College London\Desktop\Documents\GitHub\Credit-Card\src\data\clean_data.py:76: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[column] = df[column].replace({1: True, 0: False, 'true': True, 'false': False, 'True': True, 'False': False}).astype('boolean')


Total rows: 1463123
Number of rows dropped: 0
NaN values in each column:
auth_failure_reason      1461430
charge_failure_reason     446239
ip_address               1461739
card_holder_name         1461739
dtype: int64


In [4]:
df['card_bin'] = extract_bin(df, 'masked_card_number')
df['check_digit'] = extract_check_digit(df, 'masked_card_number')
df['business_delta'] = calculate_time_delta(df, 'created', 'business_id')
df['bin_delta'] = calculate_time_delta(df, 'created', 'card_bin')
df['month'] = extract_datetime_component(df, 'created', 'month')
df['day'] = extract_datetime_component(df, 'created', 'day')
df['hour'] = extract_datetime_component(df, 'created', 'hour')
df['minute'] = extract_datetime_component(df, 'created', 'minute')
df['time'] = extract_datetime_component(df, 'created', 'time')
df['mean_transactions_time'] = average_transactions_per_time(df, 'time')

nan_counts = df.isna().sum()
print("NaN values in each column:")
print(nan_counts[nan_counts > 0])
df.columns

C:\Users\User\OneDrive - Imperial College London\Desktop\Documents\GitHub\Credit-Card\src\feature_engineering\new_features.py:28: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  time_delta = df.groupby(ref_col)[time_col].diff().dt.total_seconds()


NaN values in each column:
auth_failure_reason      1461430
charge_failure_reason     446239
ip_address               1461739
card_holder_name         1461739
business_delta                 1
bin_delta                  10395
dtype: int64


Index(['business_id', 'auth_status', 'eci', 'descriptor',
       'auth_failure_reason', 'charge_failure_reason', 'amount',
       'authorized_amount', 'mid_label', 'bank_merchant_id', 'capture_amount',
       'created', 'authentication_type', 'card_bank', 'card_brand',
       'card_expiration_month', 'card_expiration_year', 'cavv', 'country',
       'currency', 'ip_address', 'masked_card_number', 'card_holder_name',
       'card_type', 'is_blocked_by_fraud', 'is_cvn_submitted',
       'should_authenticate_credit_card', 'should_settle_directly', 'card_bin',
       'check_digit', 'business_delta', 'bin_delta', 'month', 'day', 'hour',
       'minute', 'time', 'mean_transactions_time'],
      dtype='object')

In [7]:
# Create 100 bins
df['bin_detlta_bins'] = pd.cut(df['bin_delta'], bins=100)

# Calculate value counts of each bin
value_counts = df['bin_detlta_bins'].value_counts().sort_index()

# Print the value counts and bin ranges
for bin_range, count in value_counts.items():
    print(f"Bin Range: {bin_range}, Count: {count}")

Bin Range: (-22572.84, 225728.402], Count: 1427363
Bin Range: (225728.402, 451456.804], Count: 6041
Bin Range: (451456.804, 677185.206], Count: 3100
Bin Range: (677185.206, 902913.608], Count: 2338
Bin Range: (902913.608, 1128642.01], Count: 1345
Bin Range: (1128642.01, 1354370.412], Count: 1128
Bin Range: (1354370.412, 1580098.814], Count: 1223
Bin Range: (1580098.814, 1805827.216], Count: 797
Bin Range: (1805827.216, 2031555.618], Count: 911
Bin Range: (2031555.618, 2257284.02], Count: 493
Bin Range: (2257284.02, 2483012.422], Count: 404
Bin Range: (2483012.422, 2708740.824], Count: 553
Bin Range: (2708740.824, 2934469.225], Count: 456
Bin Range: (2934469.225, 3160197.627], Count: 495
Bin Range: (3160197.627, 3385926.029], Count: 324
Bin Range: (3385926.029, 3611654.431], Count: 287
Bin Range: (3611654.431, 3837382.833], Count: 217
Bin Range: (3837382.833, 4063111.235], Count: 217
Bin Range: (4063111.235, 4288839.637], Count: 194
Bin Range: (4288839.637, 4514568.039], Count: 216
Bin 

In [6]:
gc.collect()


# time delta business
# time delta card
# time delta same card but diff merchant
# various time accelerations diff of diff
# sliding time window approach

3575